In [125]:
import pandas as pd
from openai import OpenAI
import json

### Initialize OpenAI

In [53]:
openai_client = OpenAI(api_key='PUT YOUR KEY HERE')

### Read the data

In [71]:
data = pd.read_json("./intents.json")

In [72]:
data.head(50)

,intents
0,"{'tag': 'Cuts', 'patterns': ['What to do if Cu..."
1,"{'tag': 'Abrasions', 'patterns': ['how do you ..."
2,"{'tag': 'stings', 'patterns': ['How do you tre..."
3,"{'tag': 'Splinter', 'patterns': ['How to remov..."
4,"{'tag': 'Sprains', 'patterns': ['How do you tr..."
5,"{'tag': 'Strains', 'patterns': ['How do you tr..."
6,"{'tag': 'Fever', 'patterns': ['How do you trea..."
7,"{'tag': 'Nasal Congestion', 'patterns': ['How ..."
8,"{'tag': 'Cough', 'patterns': ['How to cure cou..."
9,"{'tag': 'Sore Throat', 'patterns': ['How do yo..."


In [73]:
data = data['intents']

In [74]:
len(data)

44

In [75]:
train_data = pd.DataFrame(columns=['intent','patterns','response'])

### Making the data format bit readable

In [77]:
for index in data:
        new_data = {'intent': index['tag'].lower(),'patterns': index['patterns'],'response':index['responses']}
        train_data = train_data._append(new_data, ignore_index=True)

First prompt to begin the conversation in the bot

In [141]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    health_issues_category = ['Cuts', 'Abrasions', 'stings', 'Splinter', 'Sprains', 'Strains',
    'Fever', 'Nasal Congestion', 'Cough', 'Sore Throat',
    'Gastrointestinal problems', 'Skin problems', 'Abdonominal Pain',
    'Bruises', 'Broken Toe', 'Choking', 'Wound', 'Diarrhea',
    'Frost bite', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites',
    'nose bleed', 'Pulled Muscle', 'Rectal bleeding', 'Sun Burn',
    'Testicle Pain', 'Vertigo', 'Normal Bleeding', 'Eye Injury',
    'Chemical Burn', 'Poison', 'Teeth', 'seizure', 'Head Injury',
    'Fainting', 'Headache', 'Cold', 'Rash', 'snake bite',
    'animal bite', 'Drowning', 'CPR', 'Fracture']
    
    response_format = {'issue': '_'}

    system_message = f"""
    You are an intelligent healthcare expert and your goal is to find the health issue of the patient.
    You need to ask relevant questions and understand the users issue by analysing the user's responses.
    You final objective is to find the health issue and categorise them accordingly based on the {health_issues_category} values. The values must be based on these category or you will be heavily penalised.
    The final response should in the {response_format} format with JSON value filled in the issue key. The response must be in JSON format with no additional text.
    
    To understand the users request and problems you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final health issue. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's health issues. \n
    If the health issue is unclear. Ask followup questions to understand their issue little deeper.
    categorise the health issues in that should present in the {health_issues_category}.
    Remember the instructions around the values for the different health problems.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their problems clearly. \n
    {delimiter}
    Thought 2: Ask indirect questions to understand the health issues if user is not able to clarify clearly. \n
    Aslk questions relevant to {health_issues_category} and categorise the health issues which matches the issues present in {health_issues_category}.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their problems clearly. \n
    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am facing health issues."
    Assistant: "Sorry to hear that. Can you describe little further? do you have running nose? or cough? tell me more so that I can help you"
    User: "I think i have cough but i am not sure"
    Assistant: "Thank you for providing that information. Let me see what I can help you."
    Assistant: "{response_format}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their health issues.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [122]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent healthcare expert and your goal is to find the health issue of the patient.\n    You need to ask relevant questions and understand the users issue by analysing the user\'s responses.\n    You final objective is to find the health issue and categorise them accordingly based on the [\'Cuts\', \'Abrasions\', \'stings\', \'Splinter\', \'Sprains\', \'Strains\', \'Fever\', \'Nasal Congestion\', \'Cough\', \'Sore Throat\', \'Gastrointestinal problems\', \'Skin problems\', \'Abdonominal Pain\', \'Bruises\', \'Broken Toe\', \'Choking\', \'Wound\', \'Diarrhea\', \'Frost bite\', \'Heat Exhaustion\', \'Heat Stroke\', \'Insect Bites\', \'nose bleed\', \'Pulled Muscle\', \'Rectal bleeding\', \'Sun Burn\', \'Testicle Pain\', \'Vertigo\', \'Normal Bleeding\', \'Eye Injury\', \'Chemical Burn\', \'Poison\', \'Teeth\', \'seizure\', \'Head Injury\', \'Fainting\', \'Headache\', \'Cold\', \'Rash\', \'snake bite\', \'animal bite\', \'Drowning\', \'

In [123]:
print(debug_conversation[0]['content'])


    You are an intelligent healthcare expert and your goal is to find the health issue of the patient.
    You need to ask relevant questions and understand the users issue by analysing the user's responses.
    You final objective is to find the health issue and categorise them accordingly based on the ['Cuts', 'Abrasions', 'stings', 'Splinter', 'Sprains', 'Strains', 'Fever', 'Nasal Congestion', 'Cough', 'Sore Throat', 'Gastrointestinal problems', 'Skin problems', 'Abdonominal Pain', 'Bruises', 'Broken Toe', 'Choking', 'Wound', 'Diarrhea', 'Frost bite', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'nose bleed', 'Pulled Muscle', 'Rectal bleeding', 'Sun Burn', 'Testicle Pain', 'Vertigo', 'Normal Bleeding', 'Eye Injury', 'Chemical Burn', 'Poison', 'Teeth', 'seizure', 'Head Injury', 'Fainting', 'Headache', 'Cold', 'Rash', 'snake bite', 'animal bite', 'Drowning', 'CPR', 'Fracture'] values. The values must be based on these category or you will be heavily penalised.
    The final resp

### OpenAI call for the chat completions

In [329]:
# Define a Chat Completions API call
def get_chat_completions(input):
        MODEL = 'gpt-3.5-turbo'
        chat_completion = openai_client.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)
        output = chat_completion.choices[0].message.content
        return output

In [18]:
input_prompt ='What is the capital of France?'
messages = [{'role':'user','content':input_prompt}]

In [54]:
get_chat_completions(messages)

'The capital of France is Paris.'

### Moderation check for the bad messages

In [48]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai_client.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

### Find out the solution as described by the doctors for the health issues

In [321]:
def suggest_health_solution(intent):
    data = None
    try:
        json_data = json.loads(intent)
        if 'issue' in json_data:            
            checkIntent = json_data['issue'].lower() in train_data['intent'].values
            if(checkIntent):
                #we found the intent from chatbot, lets retrieve and show the result
                data_1 = train_data[train_data['intent'] == json_data['issue'].lower()]['response']
                data = data_1[data_1.index[0]][0]
                return data
        else:
            return data
    except ValueError as e:
        print(e)
        return data

### check if dictionary is present in the text and extract it 

In [259]:
def dictionary_present(response):
    delimiter = "####"

    data_format = {"issue": "cough"}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {data_format}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format with double quotes in keys and values.
            The output should match the format as {data_format}.

            {delimiter}
            The output should contain the exact keys and values as present in the input.
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - Thank you for clarifying. Based on your symptom of running nose without sore throat or cough, it seems like you may have Nasal Congestion. {{'issue': 'Nasal Congestion'}}
            output 1: {{"issue": "Nasal Congestion"}}

            input 2: - Thank you for clarifying. Based on your symptom of fever without sore throat or cough, it seems like you may have fever. {{'issue': 'Fever'}}
            output 2: {{"issue": "Fever"}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages)

    return confirmation

In [260]:
string_dict = "Thank you for clarifying. Based on your symptom of running nose without sore throat or cough, it seems like you may have Nasal Congestion.{'issue': 'Nasal Congestion'}"""
check_dict = dictionary_present(string_dict)

In [261]:
check_dict

'{"issue": "Nasal Congestion"}'

### Dialogue management system for the bot

In [330]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + '\n')
    
    user_input = ''

    while(user_input != "exit"):
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
            
        conversation.append({"role": "user", "content": user_input})
        response_assistant = get_chat_completions(conversation)
        moderation = moderation_check(response_assistant)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
        print('Please wait while we understand the problem...\n')
        ## if json present in the text, extract it
        extract_json = dictionary_present(response_assistant)
        ## check if the issue is actually found yet
        confirmation = suggest_health_solution(extract_json)
        if(confirmation is None):
            ## issue not found, ask more questions to understand the intent
            print(response_assistant)
            conversation.append({"role": "assistant", "content": str(response_assistant)})
        else:
            ## print the health solutions
            print(str(confirmation) + '\n')

In [280]:
dialogue_mgmt_system()

'Hello! I am here to help you with your health issues. Please feel free to share any problems or concerns you may have. What health issue are you currently facing?\n'

 i have a health issue


Please wait while we understand the problem...

Sorry to hear that. Can you please describe your symptoms or health issue in more detail? Are you experiencing any specific symptoms such as cough, fever, sore throat, or any other discomfort? Please share so that I can assist you better.


 i think its high fever


Please wait while we understand the problem...

To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row.



...

Please wait while we understand the problem...

1)Keeping hydrated is absolutely vital to help 'flush' out the cold, as well as to break down congestion and keep your throat lubricated. 2)Vitamin C is extremely helpful when fighting infection, so at the first sign of a cold be sure to increase your intake by eating plenty of berries, citrus fruits, papayas, broccoli and red peppers which will help keep you protected. 3)When it comes to combating a cold,Vitamin D is essential in helping to regulate immune response.



 do you have some list of doctors?


Please wait while we understand the problem...

Sorry, I can only assist you in identifying your health issue based on the symptoms you provide. Can you please tell me if you are experiencing any other symptoms related to your health concern such as body pain, headache, chills, or any other discomfort? This will help me determine the possible health issue you might be facing.


 exit


Please wait while we understand the problem...

{'issue': 'Fever and Nasal Congestion'}


## Function calling example, it does not suit my use case at the moment

In [322]:
function_descriptions = [
    {
        "name": "get_doctors_list",
        "description": "Get list of doctors if bot can't solve a problem",
        "parameters": {
            "type": "object",
            "properties": {
                "issue": {
                    "type": "string",
                    "description": "The health issue to be solved",
                }
            },
            "required": ["issue"],
        },
    }
]

In [323]:

def get_chat_completions_functions(input):
        MODEL = 'gpt-3.5-turbo-0613'
        chat_completion = openai_client.chat.completions.create(
            model = MODEL,
            messages = input,
            functions=function_descriptions,
            function_call="auto",
            seed = 2345)
        output = chat_completion.choices[0].message
        return output

In [324]:
output = get_chat_completions_functions([{"role": "user", "content": "I think i have cuts"}])

In [325]:
output

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"issue": "cuts"\n}', name='get_doctors_list'), tool_calls=None)

In [326]:
issue = json.loads(output.function_call.arguments).get("issue")
params = json.loads(output.function_call.arguments)
type(params)
print(params)

# Call the function with arguments

chosen_function = eval(output.function_call.name)
data = chosen_function(**params)

{'issue': 'cuts'}


In [327]:
data

'Dr shahid shaikh'

In [292]:
def get_doctors_list(issue):
    return "Dr shahid shaikh"